<b> Import Required Libraries </b>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
from webdriver_manager.chrome import ChromeDriverManager

<b> Initialize the list with all the dates that need to be scraped from website </b>

In [26]:
sdate = date(2012,1,1)
edate = date(2021,11,14)
dates_dt = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
dates = []
for i in dates_dt:
    dates.append(i.strftime('%Y-%m-%d'))

<b> A function to create an empty dictionary with required fields <b>

In [18]:
def get_dict():
    dict_weather = {'date':'',
        'time':'',
        'temperature':'',
        'dew_point':'',
        'humidity':'',
        'wind':'',
        'wind_speed':'',
        'wind_gust':'',
        'pressure':'',
        'precip':'',
        'condition':''}
    
    return dict_weather

<b> Initializing the Selenium Web Driver </b>

In [28]:
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [/Users/dhruvim/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache
<ipython-input-28-c2e8b5e7b8ed>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)


<h3><b> Scraping using Beautiful Soup </b></h3>


**Beautiful Soup**
1. It is a Python library that is used for web scraping purposes to scrape the data out of HTML and XML files.
2. It creates a parse tree from page source code that can be used to extract data in a hierarchical and more readable manner.

**Procedure:-**

1. Scrape HTML Content from the weblink using Selenium Driver
2. Parse HTML Code with Beautiful Soup
3. With the help of Beautiful Soup find the required table using HTML tags 
4. Extract Text from the table
5. Save it in a temporary dictionary initialized with above defined function
6. Add the dictionary to a list and return list


In [27]:
def get_weather(url, date):
    global driver
    driver.get(url)
    time.sleep(5)
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    res = soup.find("table", {'class':'mat-table cdk-table mat-sort ng-star-inserted'})
    one_day_list = []
    try:
        for tr in res.find_all('tr')[1:]:
            tds = tr.find_all('td')
            dict_weather = get_dict()
            try:
                dict_weather['date'] = date
                dict_weather['time'] = tds[0].find('span').get_text()
                dict_weather['temperature'] = tds[1].find('span').get_text()
                dict_weather['dew_point'] = tds[2].find('span').get_text()
                dict_weather['humidity'] = tds[3].find('span').get_text()
                dict_weather['wind'] = tds[4].find('span').get_text()
                dict_weather['wind_speed'] = tds[5].find('span').get_text()
                dict_weather['wind_gust'] = tds[6].find('span').get_text()
                dict_weather['pressure'] = tds[7].find('span').get_text()
                dict_weather['precip'] = tds[8].find('span').get_text()
                dict_weather['condition'] = tds[9].find('span').get_text()

                one_day_list.append(dict_weather)
            except:
                break
    except:
        pass
            
    return one_day_list

<b> Steps for getting weather for each date and saving it in a list </b>

In [29]:
final_list = []
i = 1
for d in dates:
    print(i)
    i+=1
    url = 'https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/'+d
    temp = get_weather(url, d)
    if len(temp) < 12:
        temp = get_weather(url, d)
    final_list.extend(temp)
    print(d)

1
2020-11-08
2
2020-11-09
3
2020-11-10
4
2020-11-11
5
2020-11-12
6
2020-11-13
7
2020-11-14
8
2020-11-15
9
2020-11-16
10
2020-11-17
11
2020-11-18
12
2020-11-19
13
2020-11-20
14
2020-11-21
15
2020-11-22
16
2020-11-23
17
2020-11-24
18
2020-11-25
19
2020-11-26
20
2020-11-27
21
2020-11-28
22
2020-11-29
23
2020-11-30
24
2020-12-01
25
2020-12-02
26
2020-12-03
27
2020-12-04
28
2020-12-05
29
2020-12-06
30
2020-12-07
31
2020-12-08
32
2020-12-09
33
2020-12-10
34
2020-12-11
35
2020-12-12
36
2020-12-13
37
2020-12-14
38
2020-12-15
39
2020-12-16
40
2020-12-17
41
2020-12-18
42
2020-12-19
43
2020-12-20
44
2020-12-21
45
2020-12-22
46
2020-12-23
47
2020-12-24
48
2020-12-25
49
2020-12-26
50
2020-12-27
51
2020-12-28
52
2020-12-29
53
2020-12-30
54
2020-12-31
55
2021-01-01
56
2021-01-02
57
2021-01-03
58
2021-01-04
59
2021-01-05
60
2021-01-06
61
2021-01-07
62
2021-01-08
63
2021-01-09
64
2021-01-10
65
2021-01-11
66
2021-01-12
67
2021-01-13
68
2021-01-14
69
2021-01-15
70
2021-01-16
71
2021-01-17
72
2021-01-18
7

<b> Converting the list of dictionary to a dataframe </b>

In [31]:
df_weather = pd.DataFrame.from_dict(final_list, orient='columns')

In [32]:
df_weather

,date,time,temperature,dew_point,humidity,wind,wind_speed,wind_gust,pressure,precip,condition
0,2020-11-09,10:51 AM,69 °F,43 °F,39 °%,NW,3 °mph,0 °mph,30.30 °in,0.0 °in,Fair
1,2020-11-09,11:51 AM,75 °F,43 °F,32 °%,SSW,5 °mph,0 °mph,30.28 °in,0.0 °in,Fair
2,2020-11-09,12:51 PM,76 °F,41 °F,29 °%,SW,7 °mph,0 °mph,30.26 °in,0.0 °in,Fair
3,2020-11-09,1:51 PM,77 °F,41 °F,28 °%,SSW,7 °mph,0 °mph,30.24 °in,0.0 °in,Fair
4,2020-11-09,2:51 PM,77 °F,39 °F,25 °%,SSW,7 °mph,0 °mph,30.23 °in,0.0 °in,Fair
...,...,...,...,...,...,...,...,...,...,...,...
10188,2021-11-13,7:51 PM,45 °F,29 °F,53 °%,W,16 °mph,24 °mph,29.79 °in,0.0 °in,Fair
10189,2021-11-13,8:51 PM,44 °F,28 °F,53 °%,W,16 °mph,0 °mph,29.82 °in,0.0 °in,Fair
10190,2021-11-13,9:51 PM,44 °F,27 °F,51 °%,W,15 °mph,0 °mph,29.85 °in,0.0 °in,Fair
10191,2021-11-13,10:51 PM,44 °F,27 °F,51 °%,W,18 °mph,32 °mph,29.86 °in,0.0 °in,Fair


<b> Saving the dataframe to a CSV file so that it can be preprocessed and joined with collision dataset

In [35]:
df_weather.to_csv('weather_data.csv', mode='a', index = False, header=None)

In [37]:
driver.quit()